In [252]:
import pandas as pd
import numpy as np
import os

In [253]:
datasets_dir = 'data/'
datasets_files_names = os.listdir(datasets_dir)
datasets_files_names.remove('data.csv')
countries = [name.split('_')[0] for name in datasets_files_names]

In [254]:
for dataset_name in datasets_files_names:
    full_name = datasets_dir + dataset_name
    df = pd.read_csv(full_name)

    df = df[df['name'].apply(lambda x: len(x) > 3)]
    df.to_csv(datasets_dir + dataset_name, index=False)

In [255]:
def get_country_cities_df(country_name: str):
    file_name = country_name + "_cities.csv"

    if os.path.isfile(datasets_dir + file_name):
        return pd.read_csv(datasets_dir + file_name)
    else:
        raise Exception(f"No file with such name '{file_name}' exists")

def get_all_cities_df():
    result_df = pd.DataFrame({})

    for country in countries:
        result_df = pd.concat([result_df, get_country_cities_df(country)], ignore_index=True)

    return result_df

In [256]:
additional_names_dict = dict({
    'МСК': 'Москва',
    'СПБ': 'Санкт-Петербург',
    'Питер': 'Санкт-Петербург',
    'Питербург': 'Санкт-Петербург',
    'НСК': 'Новосибирск',
    'Сиб': 'Новосибирск',
    'Новосиб': 'Новосибирск',
    'ЕКБ': 'Екатеринбург',
    'Нижний': 'Нижний Новгород',
    'Нижнем': 'Нижний Новгород',
    'Тагил': 'Нижний Тагил',
    'АСТ': 'Астана',
    'АЛМ': 'Алматы',
    'Семей': 'Семипалатинск'
})

In [257]:
import re


def get_tokens(text: str):
    pattern = re.compile(r'\b[А-Яа-я]+\b')  # bounded word regex pattern

    return pattern.findall(text)


def calc_hamming_dist(string1: str, string2: str):
    len1 = len(string1)
    len2 = len(string2)

    min_len = min(len1, len2)
    max_len = max(len1, len2)
    delta = max_len - min_len
    dist = 0

    for i in range(min_len):
        if string1[i] != string2[i]:
            dist += 1

    dist += delta

    return dist

# search name is in lower case!
def find_by_name(search_name: str, cities_array: np.array):
    for city in cities_array:
        lower_city = city.lower()

        if search_name == lower_city:
            return city

    # check additional names dict
    for additional_name in additional_names_dict:
        if search_name == additional_name.lower():
            return additional_names_dict[additional_name]

    return None

# search name is in lower case!
def find_by_min_hamming_dist(search_name: str, cities_array: np.array):
    min_dist = 1024
    most_similar = None

    for city in cities_array:
        lower_city = city.lower()
        cur_dist = calc_hamming_dist(search_name, lower_city)

        if cur_dist < min_dist:
            min_dist = cur_dist
            most_similar = city

    for additional_name in additional_names_dict:
        additional_name_lower = additional_name.lower()
        cur_dist = calc_hamming_dist(search_name, additional_name_lower)

        if cur_dist < min_dist:
            min_dist = cur_dist
            most_similar = additional_names_dict[additional_name]   # get full city name by additional name

    return most_similar, min_dist


# 1 - use complete matching
# 2 - if don't work - see min hamming distance
def find_city(message_text: str, country: str = None) -> str:
    if country is None:
        # select all cities
        cities_df = get_all_cities_df()
    else:
        if country not in countries:
            raise Exception(f"No country with such name - '{country}', select one of supported - {countries}")
        cities_df = get_country_cities_df(country)

    city_names_list = cities_df['name'].to_numpy()
    text_tokens = get_tokens(message_text.lower())

    # 1. find by complete matching:

    for token in text_tokens:
        result = find_by_name(token, city_names_list)

        if result is not None:
            return result

    # 2. no matches on the previous step - use min hamming distance:
    min_dist = 1024
    result = None

    for token in text_tokens:
        most_similar, cur_dist = find_by_min_hamming_dist(token, city_names_list)

        if cur_dist < min_dist:
            min_dist = cur_dist
            result = most_similar

    return result

text = "Привет, нужно срочно заказать цветы на день рождения друга в Тагиле, вы сможете помочь?"
result = find_city(text, 'russia')
print(result)

Нижний Тагил


In [258]:
text = "Привет, нужно срочно заказать цветы на день рождения друга в Питере, вы сможете помочь?"
result = find_city(text, 'russia')
print(result)

Санкт-Петербург


In [259]:
test_df = pd.read_csv(datasets_dir + "data.csv")
test_df.drop(['Unnamed: 0'], axis=1, inplace=True)
test_df

,message
0,Привет! Я хочу заказать букет роз доставить в ...
1,"Привет, хочу заказать доставку цветов в Новоси..."
2,"Здравствуйте, сколько стоит доставка цветов в ..."
3,"Здравствуйте, сколько стоит доставка цветов в ..."
4,Привет! Хочу заказать цветы на день рождения п...
...,...
115,"Привет, доставите букет роз в г. екат? Срочно ..."
116,"Привет, сколько стоит доставка цветов в г. ниж..."
117,Хочу цветы в Новосибирске. Скорее быстро сдела...
118,Здравствуйте! Хочу заказать букет роз с достав...


In [260]:
messages = test_df['message'].to_numpy()
cities = []

for message in messages:
    cities.append(find_city(message))

cities = np.array(cities)
test_df['city'] = cities

In [261]:
test_df

,message,city
0,Привет! Я хочу заказать букет роз доставить в ...,Санкт-Петербург
1,"Привет, хочу заказать доставку цветов в Новоси...",Новосибирск
2,"Здравствуйте, сколько стоит доставка цветов в ...",Екатеринбург
3,"Здравствуйте, сколько стоит доставка цветов в ...",Екатеринбург
4,Привет! Хочу заказать цветы на день рождения п...,Новосибирск
...,...,...
115,"Привет, доставите букет роз в г. екат? Срочно ...",Бурея
116,"Привет, сколько стоит доставка цветов в г. ниж...",Нижний Новгород
117,Хочу цветы в Новосибирске. Скорее быстро сдела...,Новосибирск
118,Здравствуйте! Хочу заказать букет роз с достав...,Алматы


In [262]:
test_df.to_csv('data.csv')

In [263]:
string = input()
if string == "":
    print("Yes")

Yes
